# django


## 测试django orm 是否有重连机制

In [ ]:
# 测试的django 4 版本
pip3 install Django
pip3 install psycopg2-binary

In [ ]:
# 创建一个项目
django-admin startproject  myproject
python3 manage.py startapp my_app

# tree -L  2
├── manage.py
├── my_app
│   ├── admin.py
│   ├── apps.py
│   ├── migrations
│   ├── models.py     #创建model
│   ├── tests.py
│   └── views.py
├── myproject
│   ├── asgi.py
│   ├── settings.py   # 配置django文件
│   ├── urls.py
│   └── wsgi.py
└── run_db.py


In [4]:
# 配置数据库文件
# settings.py
# 添加数据库和app
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'my_app'
]

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql_psycopg2',
        'USER': 'postgres',
        'PASSWORD': '123qwe',
        'HOST': '192.168.0.59',
        'PORT': 5439,
        'NAME': 'testdb'
    },
}


In [ ]:
# 在my_app下的model.py

from django.db import models

# Create your models here.
class Info(models.Model):
    id = models.IntegerField(blank=True, null=True)
    message = models.CharField(max_length=256, blank=True, null=True)

    class Meta:
        db_table = 'info'

In [ ]:
# 在根目录下 vim run_db.py
import os
import django
import random
import time

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")
django.setup()


from django.db import close_old_connections
from my_app.models import Info

while 1:
    try:
        num = random.randint(199999, 77899931)
        # close_old_connections()
        Info.objects.filter(id=1).update(message=str(num))
        print('修改成功111')
        time.sleep(10)
    except Exception as e:
        print(e)
        time.sleep(10)

In [5]:
# 运行 run_db.py
# python3 run_db.py
# 可以看到运行结果
#修改成功111
#修改成功111
#修改成功111
#修改成功111

In [ ]:
# 1 打开终端 安装tcpkill
sudo apt install dsniff

# 2 查看pid 链接数据库的端口是5439
lsof -n -i tcp:5439
# python  8361 huahua    3u  IPv4 139249      0t0  TCP 192.168.0.119:40956->192.168.0.59:5439 (ESTABLISHED)

# 3 killtcp服务
# ens33 为网卡 可以通过ifconfig 查看
sudo tcpkill -i ens33 -9 port 40956

In [ ]:
# 查看 服务
# 显示 说明没有重连机制
connection already closed
connection already closed
connection already closed
connection already closed
connection already closed
connection already closed
connection already closed
connection already closed
connection already closed

In [ ]:
# 导入一行代码
# 每次执行修改等操作 执行close_old_connections
# 里面会去关闭久的conn 然后会执行select 1 去重新链接数据
from django.db import close_old_connections

close_old_connections()

#重新执行上面的步骤
lsof -n -i tcp:5439
# python3.6 8776 huahua    6u  IPv4 165321      0t0  TCP 192.168.0.119:59952->192.168.0.59:5439 (ESTABLISHED)
sudo tcpkill -i ens33 -9 port 59952

# 发现服务没有影响报错
# 接着查看 也建立了新链接 端口改变了
lsof -n -i tcp:5439
python3.6 8776 huahua    6u  IPv4 165511      0t0  TCP 192.168.0.119:60240->192.168.0.59:5439 (ESTABLISHED)